## Data Ingestion

In [4]:
# Import libraries and modules
import pandas as pd
import minsearch
from tqdm.auto import tqdm
from openai import OpenAI

In [6]:
df=pd.read_csv('../data/clean_data/final_data_with_IDs_new.csv')

documents = df.to_dict(orient='records')
import json
with open('../data/clean_data/documents.json', 'w') as file:
    json.dump(documents, file)

In [7]:
df=pd.read_csv('../data/clean_data/date_chunked_5s.csv')

documents = df.to_dict(orient='records')
import json
with open('../data/clean_data/documents.json', 'w') as file:
    json.dump(documents, file)

In [8]:
print(len(documents))
documents[0]


5837


{'id': 0,
 'content': 'this is the first I see an egg in bath ingredients and egg?. The plastic might even be better (especially if you have hard plastic) considering you have the cloth in between, because stainless steel could draw EMFs Your love Better anything non processed.. Sea water, an egg, sea salt, ACV, a little Urine.. .. .',
 'number of sentences': 5,
 'number of words': 56}

### Minsearch

In [9]:
# I decided not to use keywords as I discovered that it was helping with hit rate but slightly lowering it

In [10]:
index = minsearch.Index(
    text_fields=['content'],
    keyword_fields=[]
)

In [11]:
index

In [12]:
index.fit(documents)

In [13]:
index.text_fields

['content']

## RAG Flow

In [14]:
client = OpenAI()

In [15]:
query = 'How do I lose belly fat?'

In [16]:
def search(query):
    boost = {}

    results = index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=10
    )

    return results

In [17]:
search(query)

[{'id': 1366,
  'content': "what about the fat you'd gain around your belly and chubby face?. im okay with not having a six pack, i just dont wanna have lose rolls of fat on my sides and belly Are you having milkshakes?. Which one?. There are several ones.. Aajonus also said there can be a 15 degree change from person to person and although rare, right and left eye can be flipped in some people.",
  'number of sentences': 5,
  'number of words': 71},
 {'id': 2556,
  'content': 'More detoxing is necessary I wish I had that much body fat though, I frequently feel sick because I don’t have enough As long as u are overweight its fine 5-15lbs overweight is for women and 15-40lbs for men yeah.. But being fatter as a women is still recommended Funny how many people who find primal are primarily underweight people.. I am the opposite where I put on weight easily Isn’t that for men?. More fat is better Really, since I hit 60-61 kg I don’t gain any weight anymore 🤷🏻\u200d♀️ Fat = fertility Girl 

In [18]:
## LLM response
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": query}]
)

response.choices[0].message.content


"Losing belly fat involves a combination of diet, exercise, and lifestyle adjustments. Here's a comprehensive approach to help you achieve your goals:\n\n1. **Healthy Diet:**\n   - **Calorie Control:** Create a calorie deficit by consuming fewer calories than you burn. Use a calorie-tracking app to monitor your intake.\n   - **Balanced Nutrition:** Focus on whole foods such as fruits, vegetables, lean proteins, and whole grains. These are high in nutrients and help with satiety.\n   - **Limit Sugars and Refined Carbs:** Cut down on sugary snacks, drinks, and refined carbs like white bread and pasta, which can contribute to belly fat.\n   - **Increase Protein Intake:** Protein can help you feel full for longer and supports muscle maintenance. Include sources like chicken, fish, beans, and legumes.\n   - **Healthy Fats:** Consume healthy fats such as those found in avocados, nuts, and olive oil in moderation.\n\n2. **Regular Exercise:**\n   - **Cardiovascular Exercise:** Engage in activi

In [19]:
#print(_)

In [20]:
def build_prompt(query, search_results):
    prompt_template = """
    You're a primal health adviser. Answer the QUESTION based on the CONTEXT from our primal diet database.
    Use only the facts from the CONTEXT when answering the QUESTION.
    
    QUESTION: {question}
    
    CONTEXT:
    {context}
    """.strip()
    
    entry_template = """
    Chunked_Content: {Chunked_Content}
    """.strip()
    context = ""
    
    for doc in search_results:
        context = context + entry_template.format(**doc) + "\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [21]:
def build_prompt(query, search_results):
    prompt_template = """
    You're a primal health adviser. Answer the QUESTION based on the CONTEXT from our primal diet database.
    Use only the facts from the CONTEXT when answering the QUESTION.
    
    QUESTION: {question}
    
    CONTEXT:
    {context}
    """.strip()
    
    entry_template = """
    Chunked_Content: {content}
    """.strip()
    context = ""
    
    for doc in search_results:
        context = context + entry_template.format(**doc) + "\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [183]:
def llm(prompt, model='gpt-4o-mini'):
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [22]:
query='how do I lose belly fat?'
def rag(query, model='gpt-4o-mini'):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    #print(prompt)
    answer = llm(prompt, model=model)
    return answer



In [44]:
query='how to lose belly fat'
def rag(query, model='ollama3.2:1b'):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    #print(prompt)
    answer = llm(prompt, model=model)
    return answer

In [45]:
rag(query)

''

In [46]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-small")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-small")

input_text = "translate English to German: How old are you?"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids

outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0]))

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

C:\Users\user\miniconda3\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\user\.cache\huggingface\hub\models--google--flan-t5-small. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

C:\Users\user\miniconda3\Lib\site-packages\transformers\generation\utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


<pad> Wie ich er bitten?</s>


In [76]:
query='tell me about healthy food?'
def llm_flan(prompt):
    input_ids = tokenizer(prompt, return_tensors="pt",max_length=2048, truncation=True).input_ids
    #outputs = model.generate(input_ids, max_length=3048, num_beams=4, early_stopping=True)
    outputs = model.generate(
        input_ids, 
        max_new_tokens=1024,  # Set max number of new tokens
        num_beams=4,          # Keep beams, but consider reducing for diversity
        do_sample=True,       # Use sampling instead of beam search
        top_p=0.9,            # Use nucleus sampling for diversity
        early_stopping=False  # Allow the model to generate more tokens
    )
    result=tokenizer.decode(outputs[0])
    return result
    
    


In [77]:
llm_flan(query)

'<pad> Healthy foods are foods that are high in fiber, vitamins, and minerals.</s>'

In [71]:
print(_)

<pad> Take a small amount of water and a small amount of sugar.</s>


### Retrieval evaluation

In [132]:
df_questions = pd.read_csv('../data/clean_data/ground-truth-data_final.csv')

In [133]:
# print(len(df_questions))
# # I will slice it to save time
# df_questions_5000=df_questions.iloc[:1000]

In [134]:
ground_truth=df_questions.to_dict(orient='records')
len(ground_truth)

1000

In [135]:
len(ground_truth)
for q in ground_truth[0:6]:
    print(q['question'])
type(ground_truth)

What makes you say that threatening your family is not acceptable?
Why do you feel that someone needs to get a life?
Have you experienced any recent threats to your family?
What would you consider an appropriate response to someone threatening your family?
Have you taken any steps to address these threats towards your family?
Where can I find a thin silk floss that's unwaxed like silk thread?


list

In [136]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

In [137]:
def minsearch_search(query):
    boost = {}

    results = index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=10
    )

    return results

In [138]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['id']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [139]:
from tqdm.auto import tqdm

In [140]:
evaluate(ground_truth, lambda q: minsearch_search(q['question']))

  0%|          | 0/1000 [00:00<?, ?it/s]

{'hit_rate': 0.762, 'mrr': 0.5635638888888886}

In [141]:
# {'hit_rate': 0.365, 'mrr': 0.21138730158730154} # with 1000 rows
#{'hit_rate': 0.541, 'mrr': 0.34330912698412647} after chunking 20 sentence per chunk
#{'hit_rate': 0.5573333333333333, 'mrr': 0.3611481481481479} previous there are 100 sentence and also very short sentence
#{'hit_rate': 0.762, 'mrr': 0.5635638888888886} data sliced from jan 2024 to september 2024 and limit 5 sentences 
#between 100 words to 400 words per chunks

In [142]:
#{'hit_rate': 0.3724, 'mrr': 0.21373825396825424}# with 5000 rows of ground-truth questions


### Finding the best parameters

In [143]:
df_validation = df_questions[:100]
df_test = df_questions[100:200]

In [144]:
import random

def simple_optimize(param_ranges, objective_function, n_iterations=10):
    best_params = None
    best_score = float('-inf')  # Assuming we're minimizing. Use float('-inf') if maximizing.

    for _ in range(n_iterations):
        # Generate random parameters
        current_params = {}
        for param, (min_val, max_val) in param_ranges.items():
            if isinstance(min_val, int) and isinstance(max_val, int):
                current_params[param] = random.randint(min_val, max_val)
            else:
                current_params[param] = random.uniform(min_val, max_val)
        
        # Evaluate the objective function
        current_score = objective_function(current_params)
        
        # Update best if current is better
        if current_score > best_score:  # Change to > if maximizing
            best_score = current_score
            best_params = current_params
    
    return best_params, best_score

In [145]:
gt_val = df_validation.to_dict(orient='records')

In [146]:
def minsearch_search(query, boost=None):
    if boost is None:
        boost = {}

    results = index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=10
    )

    return results

In [152]:
param_ranges = {
    'content': (1.0, 3.0),
    
    
}

def objective(boost_params):
    def search_function(q):
        return minsearch_search(q['question'], boost_params)

    results = evaluate(gt_val, search_function)
    return results['mrr']

In [153]:
simple_optimize(param_ranges, objective, n_iterations=20)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

...

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

({'content': 2.3430414424052204}, 0.5565277777777778)

In [154]:
def minsearch_improved(query):
    boost = {
    
    'content': 2.3430414424052204
   
    
    }

    results = index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=10
    )

    return results

evaluate(ground_truth, lambda q: minsearch_improved(q['question']))

  0%|          | 0/1000 [00:00<?, ?it/s]

{'hit_rate': 0.762, 'mrr': 0.5635638888888886}

## RAG evaluation

In [155]:
prompt2_template = """
You are an expert evaluator for a RAG system.
Your task is to analyze the relevance of the generated answer to the given question.
Based on the relevance of the generated answer, you will classify it
as "NON_RELEVANT", "PARTLY_RELEVANT", or "RELEVANT".

Here is the data for evaluation:

Question: {question}
Generated Answer: {answer_llm}

Please analyze the content and context of the generated answer in relation to the question
and provide your evaluation in parsable JSON without using code blocks:

{{
  "Relevance": "NON_RELEVANT" | "PARTLY_RELEVANT" | "RELEVANT",
  "Explanation": "[Provide a brief explanation for your evaluation]"
}}
""".strip()

In [156]:
len(ground_truth)

1000

In [157]:
record = ground_truth[0]
record

{'id': 608,
 'question': 'What makes you say that threatening your family is not acceptable?'}

In [158]:
record = ground_truth[0]
question = record['question']
answer_llm = rag(question)
print(answer_llm)

Threatening your family is not acceptable because it can lead to harassment, fear, and emotional distress. In the context provided, there are references to threats and slander that indicate serious consequences stemming from such behavior. A person's wellbeing and safety, especially those of family members, should be prioritized, and threatening behavior undermines that security.


In [159]:
prompt = prompt2_template.format(question=question, answer_llm=answer_llm)
print(prompt)

You are an expert evaluator for a RAG system.
Your task is to analyze the relevance of the generated answer to the given question.
Based on the relevance of the generated answer, you will classify it
as "NON_RELEVANT", "PARTLY_RELEVANT", or "RELEVANT".

Here is the data for evaluation:

Question: What makes you say that threatening your family is not acceptable?
Generated Answer: Threatening your family is not acceptable because it can lead to harassment, fear, and emotional distress. In the context provided, there are references to threats and slander that indicate serious consequences stemming from such behavior. A person's wellbeing and safety, especially those of family members, should be prioritized, and threatening behavior undermines that security.

Please analyze the content and context of the generated answer in relation to the question
and provide your evaluation in parsable JSON without using code blocks:

{
  "Relevance": "NON_RELEVANT" | "PARTLY_RELEVANT" | "RELEVANT",
  "

In [160]:
import json

In [161]:
df_sample = df_questions.sample(n=200, random_state=1)

In [162]:
sample = df_sample.to_dict(orient='records')

In [163]:
evaluations = []

for record in tqdm(sample):
    question = record['question']
    answer_llm = rag(question,model='gpt-3.5-turbo') 

    prompt = prompt2_template.format(
        question=question,
        answer_llm=answer_llm
    )

    evaluation = llm(prompt)
    evaluation = json.loads(evaluation)

    evaluations.append((record, answer_llm, evaluation))

  0%|          | 0/200 [00:00<?, ?it/s]

In [164]:
df_eval = pd.DataFrame(evaluations, columns=['record', 'answer', 'evaluation'])

df_eval['id'] = df_eval.record.apply(lambda d: d['id'])
df_eval['question'] = df_eval.record.apply(lambda d: d['question'])

df_eval['relevance'] = df_eval.evaluation.apply(lambda d: d['Relevance'])
df_eval['explanation'] = df_eval.evaluation.apply(lambda d: d['Explanation'])

del df_eval['record']
del df_eval['evaluation']

In [165]:
type(evaluations[0])

tuple

In [166]:
df_eval.relevance.value_counts()

relevance
RELEVANT           126
PARTLY_RELEVANT     62
NON_RELEVANT        12
Name: count, dtype: int64

In [167]:
df_eval.relevance.value_counts(normalize=True)

relevance
RELEVANT           0.63
PARTLY_RELEVANT    0.31
NON_RELEVANT       0.06
Name: proportion, dtype: float64

In [195]:
#gpt3.5 turbo
# relevance
# RELEVANT           0.63
# PARTLY_RELEVANT    0.31
# NON_RELEVANT       0.06
# Name: proportion, dtype: float64

In [168]:
df_eval.to_csv('../data/clean_data/rag-eval-gpt-3.5-turbo.csv', index=False)

In [169]:
df_eval[df_eval.relevance == 'NON_RELEVANT'].head()

,answer,id,question,relevance,explanation
3,"No, pasteurized butter should not be consumed, as it is not recommended in the primal diet database.",3682,Is pasteurized butter from non-toxic animals safe for consumption given concerns about mad cow disease?,NON_RELEVANT,"The generated answer does not address the question regarding the safety of pasteurized butter from non-toxic animals in relation to mad cow disease. Instead, it dismisses the consumption of pasteurized butter based on an unrelated 'primal diet database' recommendation, which does not provide relevant information about mad cow disease concerns."
11,There is no information in the provided context about curing dwarfism or specific methods for someone who is 3'1 tall.,139,"What is a suggested method to help cure dwarfism, specifically for a person who is 3'1 tall?",NON_RELEVANT,"The generated answer does not provide any relevant information related to the question about suggested methods to help cure dwarfism. It states that there is no information available, which does not address the query at all."
15,I learned about the primal diet in 1982 from seeing all the evil in the world and starting with biological warfare.,4463,How did you learn about the recipe you mentioned?,NON_RELEVANT,"The generated answer discusses the primal diet and a personal experience related to it but does not address the recipe mentioned in the question. It fails to provide any relevant information about how the recipe was learned, making it irrelevant to the question asked."
34,"Non frozen, non irradiated oysters will be better for oral health compared to fruit and vegetable juices.",2181,What will be better for oral health compared to fruit and vegetable juices?,NON_RELEVANT,"The generated answer suggests oysters as better for oral health compared to fruit and vegetable juices, but it does not provide any justification or context for this claim. Additionally, it does not directly relate to the benefits or disadvantages of fruit and vegetable juices in the context of oral health, making it irrelevant to the question."
40,"No, animals do not naturally eat things like poop and maggots. There is no health perspective that supports consuming these items based on the information provided in the context.",107,"Do animals naturally eat things like poop and maggots, and is there any health perspective on this?",NON_RELEVANT,"The generated answer incorrectly states that animals do not naturally eat poop and maggots, disregarding the fact that some animals, such as dogs and certain rodents, do engage in this behavior for various reasons including nutrition. Additionally, the answer lacks a discussion of any health aspects regarding this behavior, which was a key component of the question."


In [170]:
evaluations_gpt4o_mini = []

for record in tqdm(sample):
    question = record['question']
    answer_llm = rag(question, model='gpt-4o-mini') 

    prompt = prompt2_template.format(
        question=question,
        answer_llm=answer_llm
    )

    evaluation = llm(prompt)
    evaluation = json.loads(evaluation)
    
    evaluations_gpt4o_mini.append((record, answer_llm, evaluation))

  0%|          | 0/200 [00:00<?, ?it/s]

In [171]:
df_eval = pd.DataFrame(evaluations_gpt4o_mini, columns=['record', 'answer', 'evaluation'])

df_eval['id'] = df_eval.record.apply(lambda d: d['id'])
df_eval['question'] = df_eval.record.apply(lambda d: d['question'])

df_eval['relevance'] = df_eval.evaluation.apply(lambda d: d['Relevance'])
df_eval['explanation'] = df_eval.evaluation.apply(lambda d: d['Explanation'])

del df_eval['record']
del df_eval['evaluation']

In [172]:
df_eval.relevance.value_counts(normalize=True)

relevance
RELEVANT           0.705
PARTLY_RELEVANT    0.255
NON_RELEVANT       0.040
Name: proportion, dtype: float64

In [194]:
# gpt4 mini
# relevance
# RELEVANT           0.705
# PARTLY_RELEVANT    0.255
# NON_RELEVANT       0.040
# Name: proportion, dtype: float64

In [173]:
df_eval.to_csv('../data/clean_data/rag-eval-gpt_4o_mini.csv', index=False)

In [42]:
import subprocess
def llm(prompt,model="llama3.2:1b"):
    # Command to run Ollama with the desired model and prompt
    command = ["ollama", "run", model, prompt]

    # Run the command and capture the output
    result = subprocess.run(command, capture_output=True, text=True, encoding='utf-8')

    # Return the result
    return result.stdout

In [43]:
llm('how to lose belly fat')

"Losing belly fat can be challenging, but it's achievable with a combination of healthy diet, regular exercise, and lifestyle changes. Here are some tips to help you shed those extra pounds:\n\n**Diet:**\n\n1. **Eat more protein**: Protein takes more energy to digest, which can help increase your metabolism and burn belly fat. Aim for 0.8-1 gram of protein per pound of body weight from sources like lean meats, fish, eggs, tofu, and legumes.\n2. **Incorporate healthy fats**: Nuts, seeds, avocados, and olive oil are rich in healthy fats that can help reduce inflammation and improve insulin sensitivity.\n3. **Focus on whole foods**: Eat plenty of fruits, vegetables, whole grains, and lean proteins to provide your body with the necessary nutrients for weight loss and overall health.\n4. **Reduce sugar intake**: Consuming high amounts of sugar can lead to increased belly fat storage. Limit sugary drinks, sweets, and refined carbohydrates.\n5. **Stay hydrated**: Drink plenty of water through

In [5]:
evaluations_llama3_2 =[]
 
for record in tqdm(sample):
    question = record['question']
    answer_llm = rag(question, model='llama3.2') 

    prompt = prompt2_template.format(
        question=question,
        answer_llm=answer_llm
    )

    evaluation = llm(prompt)
    evaluation = json.loads(evaluation)
    
    evaluations_llama3_2.append((record, answer_llm, evaluation))


NameError: name 'sample' is not defined

In [ ]:
df_eval = pd.DataFrame(evaluations_gpt4o, columns=['record', 'answer', 'evaluation'])

df_eval['id'] = df_eval.record.apply(lambda d: d['id'])
df_eval['question'] = df_eval.record.apply(lambda d: d['question'])

df_eval['relevance'] = df_eval.evaluation.apply(lambda d: d['Relevance'])
df_eval['explanation'] = df_eval.evaluation.apply(lambda d: d['Explanation'])

del df_eval['record']
del df_eval['evaluation']

In [186]:
df_eval.relevance.value_counts()

relevance
RELEVANT           141
PARTLY_RELEVANT     51
NON_RELEVANT         8
Name: count, dtype: int64

In [187]:
df_eval.relevance.value_counts(normalize=True)

relevance
RELEVANT           0.705
PARTLY_RELEVANT    0.255
NON_RELEVANT       0.040
Name: proportion, dtype: float64

In [ ]:
df_eval.to_csv('../data/clean_data/rag-eval-llama3_2.csv', index=False)

In [188]:
evaluations_gpt4o = []

for record in tqdm(sample):
    question = record['question']
    answer_llm = rag(question, model='gpt-4o') 

    prompt = prompt2_template.format(
        question=question,
        answer_llm=answer_llm
    )

    evaluation = llm(prompt)
    evaluation = json.loads(evaluation)
    
    evaluations_gpt4o.append((record, answer_llm, evaluation))

  0%|          | 0/200 [00:00<?, ?it/s]

In [196]:
df_eval = pd.DataFrame(evaluations_gpt4o, columns=['record', 'answer', 'evaluation'])

df_eval['id'] = df_eval.record.apply(lambda d: d['id'])
df_eval['question'] = df_eval.record.apply(lambda d: d['question'])

df_eval['relevance'] = df_eval.evaluation.apply(lambda d: d['Relevance'])
df_eval['explanation'] = df_eval.evaluation.apply(lambda d: d['Explanation'])

del df_eval['record']
del df_eval['evaluation']

In [197]:
df_eval.relevance.value_counts()

relevance
RELEVANT           137
PARTLY_RELEVANT     55
NON_RELEVANT         8
Name: count, dtype: int64

In [198]:
df_eval.relevance.value_counts(normalize=True)

relevance
RELEVANT           0.685
PARTLY_RELEVANT    0.275
NON_RELEVANT       0.040
Name: proportion, dtype: float64

gpt 4o  
relevance  
RELEVANT           0.685  
PARTLY_RELEVANT    0.275  
NON_RELEVANT       0.040  
Name: proportion, dtype: float64

In [199]:
df_eval.to_csv('../data/clean_data/rag-eval-gpt4o.csv', index=False)

In [200]:
import pandas as pd

In [202]:
df=pd.read_csv('../data/clean_data/rag-eval-gpt4o.csv')

In [203]:
df['relevance']

0      PARTLY_RELEVANT
1      PARTLY_RELEVANT
2             RELEVANT
3      PARTLY_RELEVANT
4             RELEVANT
5             RELEVANT
6             RELEVANT
7         NON_RELEVANT
8      PARTLY_RELEVANT
9      PARTLY_RELEVANT
10     PARTLY_RELEVANT
11            RELEVANT
12            RELEVANT
13     PARTLY_RELEVANT
14            RELEVANT
15     PARTLY_RELEVANT
16            RELEVANT
17            RELEVANT
18            RELEVANT
19            RELEVANT
20     PARTLY_RELEVANT
21     PARTLY_RELEVANT
22            RELEVANT
23            RELEVANT
24            RELEVANT
25            RELEVANT
26            RELEVANT
27     PARTLY_RELEVANT
28     PARTLY_RELEVANT
29            RELEVANT
30            RELEVANT
31            RELEVANT
32            RELEVANT
33     PARTLY_RELEVANT
34     PARTLY_RELEVANT
35            RELEVANT
36            RELEVANT
37            RELEVANT
38            RELEVANT
39     PARTLY_RELEVANT
40            RELEVANT
41     PARTLY_RELEVANT
42            RELEVANT
43     PART

In [204]:
pd.set_option("display.max_rows", None)
pd.set_option('display.max_colwidth', None)
df[df['relevance']=='PARTLY_RELEVANT'] 

,answer,id,question,relevance,explanation
0,"No, not everything in Europe comes in glass containers. According to the context, there's an option to pay extra for milk in glass containers, but it is not the standard practice for all products.",5385,"In Europe, does everything including milk come in glass containers?",PARTLY_RELEVANT,The generated answer addresses the question by stating that not everything comes in glass containers and mentions the option for milk in glass but does not provide comprehensive information about the overall context of container usage in Europe.
1,"The context does not provide specific information on whether eating raw foods saves time compared to preparing processed foods. However, the preparation time could be reduced simply because cooking is not involved, but this is not explicitly mentioned in the provided data.",871,Can eating raw foods save time compared to more processed food preparation?,PARTLY_RELEVANT,"The generated answer acknowledges the potential time savings of eating raw foods due to not involving cooking, which is relevant to the question. However, it states that the context does not provide explicit information and does not directly answer the question, leading to a classification of partly relevant."
3,"According to the context provided, pasteurized butter from non-toxic animals is mentioned in relation to mad cow disease. The context suggests that mad cow disease was associated with consuming organs from toxic animals, but it does not directly address pasteurized butter from non-toxic animals. However, another part of the context contains a direct statement advising against consuming pasteurized butter. Therefore, based on the information given, it is advised not to eat pasteurized butter.",3682,Is pasteurized butter from non-toxic animals safe for consumption given concerns about mad cow disease?,PARTLY_RELEVANT,"The generated answer addresses the question about pasteurized butter and mad cow disease by referencing the safety of pasteurized butter from non-toxic animals. However, it ultimately contradicts the initial implication of safety by stating it is advised not to eat pasteurized butter, which introduces ambiguity. Thus, while there is some relevance, it does not provide a clear, conclusive answer to the safety of the specified product."
8,"Based on the provided context, there is no specific mention of any cheese makers from Thailand in the group who can offer advice. There is a mention of someone from Austria living in Thailand, but it does not specify they are a cheese maker.",4314,Are there any cheese makers from Thailand in this group who can offer advice?,PARTLY_RELEVANT,"The generated answer acknowledges the absence of specific cheese makers from Thailand in the group and does mention a related individual from Austria living in Thailand. However, it does not directly address whether any Thai cheese makers can offer advice, making it somewhat relevant but ultimately incomplete."
9,"Feeding your cat raw beef twice a week might provide some benefits but it's not guaranteed to extend his lifespan. Some people have observed benefits from feeding cats raw meat even once a month. However, there are potential concerns about detoxification or digestive issues if the cat does not handle raw meat well. Ensuring the meat is organic could be important according to the context. Consulting with a veterinarian is recommended to address any specific health concerns and to determine the most appropriate diet for longevity and health.",4019,Will feeding my cat raw beef twice a week help extend his lifespan?,PARTLY_RELEVANT,"The generated answer acknowledges the potential benefits of feeding raw beef to a cat and points out the uncertainty regarding its impact on lifespan. It also highlights possible health concerns and emphasizes the importance of consulting a veterinarian. However, it does not directly answer the question about extending lifespan, making it partially relevant."
10,"To

In [13]:
df.relevance.unique()

array(['RELEVANT', 'PARTLY_RELEVANT', 'NON_RELEVANT'], dtype=object)